In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

cdl_df = pd.read_csv("Credit_card_label.csv")
cd_df = pd.read_csv("Credit_card.csv")
cd_df.head(3)

In [5]:
cd_df.isnull().sum()

Below I further cleanse null values including for GENDER and Type_Occupation

In [6]:
from sklearn.impute import SimpleImputer

imputer =  SimpleImputer(missing_values=np.nan, strategy='mean')
cols_to_impute_numeric = ['Annual_income','Birthday_count']
cd_df[cols_to_impute_numeric] = imputer.fit_transform(cd_df[cols_to_impute_numeric])
    

imputer =  SimpleImputer(missing_values=np.nan, strategy = 'most_frequent' )
cols_to_impute_string = ['GENDER','Type_Occupation']
cd_df[cols_to_impute_string] = imputer.fit_transform(cd_df[cols_to_impute_string])
cd_df.isnull().sum()

In [7]:
df = pd.merge(cd_df, cdl_df, on='Ind_ID', how="outer")
df = df.drop(columns=['Ind_ID','Mobile_phone'], axis=1)
df.head(3)

In [8]:
number_columns = ['GENDER', 'Car_Owner', 'Propert_Owner']
other_columns = ['Type_Income', 'EDUCATION', 'Marital_status', 'Housing_type', 'Type_Occupation']

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in number_columns:
    df[i] = le.fit_transform(df[i])
df.head(3)

I expanded my get dummies to more columns

In [10]:
df = pd.get_dummies(df, columns = other_columns, dtype=int)
df.head(3)

Part 2

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [12]:
df["label"].value_counts()

In [13]:
X = df.drop('label', axis = 1)
Y = df['label']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.3,
                                                    random_state=42,
                                                    stratify=Y)

In [14]:
from imblearn.under_sampling import ClusterCentroids
rus = ClusterCentroids(random_state=42)
X_resampled, Y_resampled = rus.fit_resample(X_train, Y_train)
model = LogisticRegression(random_state=42)
model.fit(X_resampled, Y_resampled)

In [15]:
Y_pred = model.predict(X_test)
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_estimator(model, X_test, Y_test)

Undersampling cooked becuase it takes away from the majority case making the minority case to majority case bias less

In [ ]:
47/(47+6)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier

X = df.drop('label', axis = 1)
Y = df['label']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.3,
                                                    random_state=42)

from sklearn.preprocessing import StandardScaler
S = StandardScaler()
X_train = S.fit_transform(X_train)
X_test = S.fit_transform(X_test)
knn = KNeighborsRegressor(n_neighbors=12)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
Y_pred

In [ ]:
from sklearn.svm import SVC
model_SVC = SVC(kernel = "linear", random_state=4)
model_SVC.fit(X_train, Y_train)

In [ ]:
model_perf = model_SVC.predict(X_test)
print(model_perf)

Logistic regrssion to SVC the predict there are less 1 values with SVC

In [ ]:

ConfusionMatrixDisplay.from_estimator(model_SVC, X_test, Y_test)

part 3

In [21]:
from sklearn.metrics import roc_curve, auc

logistic_fpr, logistic_tpr, threshold = roc_curve(Y_test, Y_pred)
auc_logistic =auc(logistic_fpr, logistic_tpr)

In [ ]:
from sklearn.datasets import make_classification
from sklearn.metrics import RocCurveDisplay
RocCurveDisplay.from_estimator(model_SVC, X_test, Y_test)

plt.show

In [23]:
auc_val = auc(logistic_fpr, logistic_tpr)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y, model.predict_proba(X)[:,1])

In [ ]:
roc_auc_score(Y, model.decision_function(X))

Outlier detection which is also called anomially detection is the process of identifying data points that possess an extreme value compared to the rest of the distribution. It can be used for a variety of application like detecting bank fraud or cyber security attacks. When desciding what method to choose you first have to look at the datas modality, shape, and size. From there you can decide if you should use box plots, isolation forsest, or oneclassSVM.